## Reexecução

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## File Paths & Magic Functions

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

inPath = '/home/lzirondi/AlexandreMartins'
outPath = '/home/lzirondi'
corpusPath = '/home/lzirondi/Snorkel_Farmacovigilancia/Scripts/old/Datasets/Processed/Tweets Anotados.tsv'
LIWCpath = '/home/lzirondi/Github/snorkel/Scripts/'



#Calculando o número de linhas no corpus
with open(corpusPath) as tsv:
    s = tsv.readlines()
    n_docs = len(s)
    s = None

## Nova execução

In [2]:
#Fazendo o link simbólico para o acesso ao banco de dados e definindo o banco como o PSQL
import os
os.system("ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432")
os.environ['SNORKELDB'] = 'postgres:///lzirondi'

#O import do SnorkelSession SEMPRE vai ter que estar após definir o BD nas variaveis de ambiente do SO, ao contrário ele defaulta para SQLite
from snorkel import SnorkelSession
session = SnorkelSession()

## Gerando Sentenças

In [3]:
from snorkel.parser import TSVDocPreprocessor
doc_preprocessor = TSVDocPreprocessor(corpusPath, max_docs=n_docs)

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy(lang='pt'))

%time corpus_parser.apply(doc_preprocessor, count=n_docs, parallelism=16)

from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/home/lzirondi/anaconda3/envs/snorkel-extraction/lib/python3.6/site-packages/pt_core_news_sm
-->
/home/lzirondi/anaconda3/envs/snorkel-extraction/lib/python3.6/site-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
Clearing existing...
100%|██████████| 5984/5984 [05:42<00:00, 17.48it/s]CPU times: user 5.31 s, sys: 1.08 s, total: 6.38 s
Wall time: 5min 43s
Documents: 5984
Sentences: 8280



## Preparando os Dicionários

In [4]:
import Preprocessing as prep


util = prep.Util(inPath, outPath, LIWCpath, True)


substancias = util.subsToList()
eventos = util.eventosToList()

Arquivo Tweets Anotados.txt copiado com sucesso.
Arquivo Eventos.txt copiado com sucesso.
Arquivo Substâncias2-br-gazette.txt copiado com sucesso.
Arquivo Remédios2-br-gazette.txt copiado com sucesso.
Arquivo Substâncias-br-gazette.txt copiado com sucesso.
Arquivo Remédios-br-gazette.txt copiado com sucesso.
Arquivo EventosAdversos-gazette.txt copiado com sucesso.
Tsv do Corpus criado com sucesso.
Dicionário de Substâncias gerado com sucesso.
Dicionário de Eventos gerado com sucesso.


## Gerando a Classe de Candidatos

In [5]:
from snorkel.models import candidate_subclass

relation = candidate_subclass('Relation', ['Substância', 'Evento_Adverso'])

## Extraindo Candidatos do Corpus

In [6]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DictionaryMatch

ngrams = Ngrams(n_max=7)
substancia_matcher = DictionaryMatch(d=substancias, longest_match_only=True, ignore_case=True)
eventos_matcher = DictionaryMatch(d=eventos, longest_match_only=True, ignore_case=True)
cand_extractor = CandidateExtractor(relation, [ngrams, ngrams], [substancia_matcher, eventos_matcher], symmetric_relations=True)

from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)

In [8]:
print("Dev_sents " + str(len(dev_sents)))
print("test_sents " + str(len(test_sents)))
print("train_sents " + str(len(train_sents)))

Dev_sents 810
test_sents 811
train_sents 6659


In [9]:
#

%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    print(i)
    cand_extractor.apply(sents, split=i, parallelism = 16)
    print("Number of candidates:", session.query(relation).filter(relation.split == i).count())

  0%|          | 0/6659 [00:00<?, ?it/s]CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
0
Clearing existing...
Running UDF...
  0%|          | 0/810 [00:00<?, ?it/s]Number of candidates: 5156
1
Clearing existing...
Running UDF...
  0%|          | 0/811 [00:00<?, ?it/s]Number of candidates: 652
2
Clearing existing...
Running UDF...
100%|██████████| 811/811 [00:11<00:00, 72.00it/s]Number of candidates: 626



In [10]:
print('0  ' + str(session.query(relation).filter(relation.split == 0).count() ))
print('1  ' + str(session.query(relation).filter(relation.split == 1).count() ))
print('2  ' + str(session.query(relation).filter(relation.split == 2).count() ))

print('Total ' + str(session.query(relation).filter(relation.split == 0).count() + session.query(relation).filter(relation.split == 1).count() + session.query(relation).filter(relation.split == 2).count()))

candi0 = session.query(relation).filter(relation.split == 0).all()[0]
candi1 = session.query(relation).filter(relation.split == 1).all()[1]
candi2 = session.query(relation).filter(relation.split == 2).all()[2]

print(candi0)
print(candi1)
print(candi2)
'''
6k sentenças 10k candidatos
'''

0  5156
1  652
2  626
Total 6434
Relation(Span("b'dorflex'", sentence=111812, chars=[12,18], words=[3,3]), Span("b'dor'", sentence=111812, chars=[66,68], words=[13,13]))
Relation(Span("b'ibuprofeno'", sentence=111626, chars=[0,9], words=[0,0]), Span("b'dor'", sentence=111626, chars=[16,18], words=[2,2]))
Relation(Span("b'omeprazol'", sentence=112482, chars=[34,42], words=[6,6]), Span("b'azia'", sentence=112482, chars=[27,30], words=[4,4]))


'\n6k sentenças 10k candidatos\n'

In [30]:
#print(session.query(relation).filter(relation.split == 0).first())

print(session.query('Candidate').filter(relation.split == 0).all())

('(51924,relation,0)',), ('(51925,relation,0)',), ('(51926,relation,0)',), ('(51927,relation,0)',), ('(51928,relation,0)',), ('(51929,relation,0)',), ('(51930,relation,0)',), ('(51931,relation,0)',), ('(51932,relation,0)',), ('(51933,relation,0)',), ('(51934,relation,0)',), ('(51935,relation,0)',), ('(51936,relation,0)',), ('(51937,relation,0)',), ('(51938,relation,0)',), ('(51939,relation,0)',), ('(51940,relation,0)',), ('(51941,relation,0)',), ('(51942,relation,0)',), ('(51943,relation,0)',), ('(51944,relation,0)',), ('(51945,relation,0)',), ('(51946,relation,0)',), ('(51947,relation,0)',), ('(51948,relation,0)',), ('(51949,relation,0)',), ('(51950,relation,0)',), ('(51951,relation,0)',), ('(51952,relation,0)',), ('(51953,relation,0)',), ('(51954,relation,0)',), ('(51955,relation,0)',), ('(51956,relation,0)',), ('(51957,relation,0)',), ('(51958,relation,0)',), ('(51959,relation,0)',), ('(51960,relation,0)',), ('(51961,relation,0)',), ('(51962,relation,0)',), ('(51963,relation,0)',), 

# Execução Incompleta

## Gerando um .tsv dos candidatos

In [ ]:
cands = session.query(relation).filter(relation.split == 0).all()

f = open('/home/lzirondi/Github/snorkel/Scripts/Datasets/Processed/Candidatos.tsv', 'w', encoding='UTF-8')
f.write('Sentença\t Span_1 \t Span_2 \n')

for candidate in cands:   
    
    parent = candidate.get_parent()
    parent = parent._asdict()
    parent = parent.get('text')
    parent = parent.replace('\n', '')
    
    
    contexts = candidate.get_contexts()
    span_list = [contexts[0].get_span(), contexts[1].get_span()]   
    
    
    f.write(parent + '\t' + span_list[0] + '\t' + span_list[1] + '\n' )
    
    
f.close()

## Random Stuff

In [ ]:
#par = candi.get_parent()
#dict1 = par._asdict()

#contexts = candi.get_contexts()

#spans = [contexts[0], contexts[1]]

#a = contexts[0]

#for candidate in cands:
#    print(candidate.get_contexts()[0].get_span())
#print(a.get_span())


#object_methods = [method_name for method_name in dir(a) if callable(getattr(a, method_name))]

#print(object_methods)
#cands = session.query(causas).filter(causas.split == 0).all()

#cands2 = session.query(causas).filter(causas.split==0).all()
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

print(session.query(causas).filter(causas.split==0).all())

#print(cands)
#print(cands[527])

#print(cands2[527])


from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)



In [ ]:
#print(type(parent))
#print(type(contexts_list[0]))
#print(type(contexts_list[0]))
#print(type(cands[0]))
    

#print(cands[0].get_cids())
#print(type(cands[0].get_contexts()))
#print(cands[0].get_parent())

#object_methods = [method_name for method_name in dir(cands[0]) if callable(getattr(cands[0], method_name))]

#print(object_methods)

#a = cands[0].get_contexts()

#object_methods = [method_name for method_name in dir(contexts[0]) if callable(getattr(contexts[0], method_name))]

#print(object_methods)

#print(type(a))

#f = open('/home/lzirondi/Github/snorkel/Scripts/Datasets/Processed/Candidatos.txt', 'w', encoding='UTF-8')
#for span in cands:
#    print(str(span))


In [ ]:
def liwcInCandidate(c):
    return 1 if se.contains(c.get_parent().words) else 0

    

for c in session.query(relation).filter(relation.split == 1).all():
    if liwcInCandidate(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))

In [ ]:
from snorkel.viewer import SentenceNgramViewer


#SentenceNgramViewer(labeled, session)


cands = session.query(relation).filter(relation.split == 0).all()

print(cands[0])
SentenceNgramViewer(cands, session)

#c = cands[0]

#print(c.get_parent().words)